In [9]:
%%writefile CombiningTree.go

package main

import (
	"container/list"
	"fmt"
	"math"
	"sync"
	"time"
)

type CStatus int

const (
	IDLE CStatus = iota
	FIRST
	SECOND
	RESULT
	ROOT
)

type Node struct {
	parent      *Node
	cStatus     CStatus
	locked      bool
	result      int
	firstValue  int
	secondValue int
	cond sync.Cond
}

func newNode(myParent *Node) Node {
	n := Node{parent: myParent}
	n.cStatus = IDLE
	n.locked = false
	n.result = 0
	n.firstValue = 0
	n.secondValue = 0
	n.cond = sync.Cond{}
	n.cond.L = &sync.Mutex{}
	return n
}

func (node *Node) precombine() bool {
	node.cond.L.Lock()
	defer node.cond.L.Unlock()
	for node.locked {
		node.cond.Wait()
	}
	switch node.cStatus {
	case IDLE:
		node.cStatus = FIRST
		return true
	case FIRST:
		node.locked = true
		node.cStatus = SECOND
		return false
	case ROOT:
		return false
	default:
		fmt.Println("unexpected Node state in precombine", node.cStatus)
		return false // error
	}
}

func (node *Node) combine(combined int) int {
	node.cond.L.Lock()
	defer node.cond.L.Unlock()
	for node.locked {
		node.cond.Wait()
	}
	node.locked = true
	node.firstValue = combined
	switch node.cStatus {
	case FIRST:
		return node.firstValue
	case SECOND:
		return node.firstValue + node.secondValue
	default:
		fmt.Println("unexpected Node state in combine", node.cStatus)
		return -1 // error

	}
}

func (node *Node) op(combined int) int {
	node.cond.L.Lock()
	defer node.cond.L.Unlock()
	switch node.cStatus {
	case ROOT:
		prior := node.result
		node.result += combined
		return prior
	case SECOND:
		node.secondValue = combined
		node.locked = false
		node.cond.Broadcast()  // wake up waiting threads
		for node.cStatus != RESULT {
			node.cond.Wait()
		}
		node.locked = false
		node.cond.Broadcast()
		node.cStatus = IDLE
		return node.result
	default:
		fmt.Println("unexpected Node state in op", node.cStatus)
		return -1 // error

	}
}

func (node *Node) distribute(prior int) {
	node.cond.L.Lock()
	defer node.cond.L.Unlock()
	switch node.cStatus {
	case FIRST:
		node.cStatus = IDLE
		node.locked = false
	case SECOND:
		node.result = prior + node.firstValue
		node.cStatus = RESULT
	default:
		fmt.Println("unexpected Node state in distribute", node.cStatus)
	}
	node.cond.Broadcast()
}

type CombinningTree struct {
	nodes []Node
	leaf  []*Node
}

func newTree(width int) CombinningTree {

	tree := CombinningTree{}
	tree.nodes = make([]Node, 2*width-1)
	tree.leaf = make([]*Node, width)
	length := 2*width - 1
	tree.nodes[0] = Node{
		parent:      nil,
		cStatus:     ROOT,
		locked:      false,
		result:      0,
		firstValue:  0,
		secondValue: 0,
		cond: sync.Cond{},
	}
	tree.nodes[0].cond.L = &sync.Mutex{}
	for i := 1; i < length; i++ {
		tree.nodes[i] = newNode(&tree.nodes[(i-1)/2])
	}
	for i := 0; i < width; i++ {
		tree.leaf[i] = &tree.nodes[length-i-1]
	}
	return tree
}

func (tree *CombinningTree) getAndIncrement(id int) int {
	stack := list.New()
	myLeaf := tree.leaf[id%len(tree.leaf)]
	node := myLeaf
	for node.precombine() {
		node = node.parent
	}
	stop := node
	combined := 1
	for node = myLeaf; node != stop; node = node.parent {
		combined = node.combine(combined)
		stack.PushBack(node)
	}
	prior := stop.op(combined)
	for stack.Len() > 0 {
		node = stack.Remove(stack.Back()).(*Node)
		node.distribute(prior)
	}
	return prior

}

var TH int
var NUM int

func main() {
	TH = 12
	NUM = 100
	width := int(math.Ceil(float64(TH / 2)))
	tree := newTree(width)
	fmt.Println(width, "-leaves Combining tree.")
	fmt.Println("Starting", TH ,"threads doing increments ...")
	var wg sync.WaitGroup

	start := time.Now()
	for index := 0; index < TH; index++ {
		wg.Add(1)
		go func(id int) {
			defer wg.Done()
			s := time.Now()
			for i := 0; i < NUM; i++ {
				tree.getAndIncrement(id)
			}
			e := time.Now()
			fmt.Println(id, "done in", e.Sub(s))
		}(index)
	}
	wg.Wait()
	stop := time.Now()
	diff := stop.Sub(start)
	fmt.Println("Total:", tree.nodes[0].result)
	fmt.Println("Total time:", diff)

}


Overwriting CombiningTree.go


In [12]:
!go run CombiningTree.go

6 -leaves Combining tree.
Starting 12 threads doing increments ...
0 done in 527.3碌s
5 done in 527.3碌s
10 done in 527.3碌s
4 done in 527.3碌s
11 done in 527.3碌s
6 done in 527.3碌s
7 done in 527.3碌s
1 done in 527.3碌s
9 done in 527.3碌s
8 done in 527.3碌s
2 done in 527.3碌s
3 done in 1.0596ms
Total: 1200
Total time: 1.0596ms


In [2]:
%%writefile CounterNetwork.go

package main

import (
	"fmt"
	"math/rand"
	"sync"
	"time"
)

type Balancer struct {
	toggle bool
	lock   sync.Mutex
}

func (b *Balancer) balancerTraverse() int {
	b.lock.Lock()
	defer b.lock.Unlock()
	defer func() { b.toggle = !b.toggle }()
	if b.toggle {
		return 0
	} else {
		return 1
	}
}

type Merger struct {
	width int
	layer []Balancer
	half  []Merger
}

func newMerger(myWidth int) Merger {
	m := Merger{}
	m.width = myWidth
	m.layer = make([]Balancer, myWidth/2)
	for i := 0; i < m.width/2; i++ {
		m.layer[i] = Balancer{toggle: true}
	}
	if m.width > 2 {
		m.half = []Merger{newMerger(myWidth / 2), newMerger(myWidth / 2)}
	}
	return m
}

func (m *Merger) mergerTraverse(input int) int {
	output := 0
	if m.width <= 2 {
		return m.layer[0].balancerTraverse()
	}
	if input < m.width/2 {
		output = m.half[input%2].mergerTraverse(input / 2)
	} else {
		output = m.half[1-(input%2)].mergerTraverse(input / 2)
	}
	return (2 * output) + m.layer[output].balancerTraverse()

}

type Bitonic struct {
	half   []Bitonic
	merger Merger
	width  int
}

func newBitonic(myWidth int) Bitonic {
	bit := Bitonic{}
	bit.width = myWidth
	bit.merger = newMerger(myWidth)
	if myWidth > 2 {
		bit.half = []Bitonic{newBitonic(myWidth / 2), newBitonic(myWidth / 2)}
	}
	return bit
}

func (bit *Bitonic) bitonicTraverse(input int) int {
	output := 0
	subnet := input / (bit.width / 2)
	if bit.width > 2 {
		output = bit.half[subnet].bitonicTraverse(input / 2)
	}
	if input >= bit.width/2 {
		return bit.merger.mergerTraverse(bit.width/2 + output)
	} else {
		return bit.merger.mergerTraverse(output)
	}
}

func main() {
	width := 4
	counters := make([]int, width)
	bitonic := newBitonic(width)
	tokenCount := 1000
	tokens := make([]int, tokenCount)
	var wg sync.WaitGroup

	for i := 0; i < tokenCount; i++ {
		tokens[i] = rand.Intn(width)
	}

	start := time.Now()
	for idx := 0; idx < tokenCount; idx++ {
		wg.Add(1)
		go func(i int) {
			defer wg.Done()
			counters[bitonic.bitonicTraverse(tokens[i])] += 1
		}(idx)
	}

	wg.Wait()
	stop := time.Now()
	diff := stop.Sub(start)

	for i := 0; i < width; i++ {
		fmt.Println("output: ", i, ", Count: ", counters[i])
	}
	fmt.Println("Total time to traverse the network: ", diff)
}


Overwriting CounterNetwork.go


In [3]:
!go run CounterNetwork.go

output:  0 , Count:  250
output:  1 , Count:  250
output:  2 , Count:  250
output:  3 , Count:  250
Total time to traverse the network:  1.1012ms
